# LangChain Agents Bootcamp - Financial Analysis with Gemini

Complete guide to building production-ready agents with memory, middleware, and streaming.

## Overview

This notebook covers:
- **Short-term Memory**: Persist conversation state with SQLite
- **Built-in Middleware**: 7 production patterns (summarization, limits, PII, etc.)
- **Context Engineering**: Manual and automatic context management
- **Structured Output**: Type-safe agent responses
- **Streaming Modes**: Real-time updates (`messages`, `updates`, `values`)
- **Context Caching**: Cost-efficient large document analysis

## Setup

Initialize model and tools for financial analysis.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from scripts import base_tools

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
# model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

system_prompt = """You are a financial analyst specializing in tech stocks.
Provide data-driven analysis with clear insights."""

## 1. Basic Agent

Create a simple agent with tools but no memory.

In [3]:
agent = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt
)

response = agent.invoke({
    "messages": [HumanMessage("What's Apple's current stock price?")]
})

print(response["messages"][-1].text)

Apple Inc. (AAPL) is currently trading at $277.82.


In [4]:
response

{'messages': [HumanMessage(content="What's Apple's current stock price?", additional_kwargs={}, response_metadata={}, id='19585788-03e1-4c50-85e4-bc1b0bce3d0c'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'web_search', 'arguments': '{"query": "Apple stock price"}'}, '__gemini_function_call_thought_signatures__': {'7917f278-7478-4921-9f3c-b8faa11b783b': 'Cv4BAXLI2nyGsA2CPj+kXYDe4okaL+mR81KbB+ZnURmuTO+5CMzZXT6FKGvbv3EAULZkqc5Id5j+vz5JfUYrtEz1Fz5w5fnrH/Uh+kxsE/6SMx12yD2C1SKR6xjCNqOjbkC6W8fy7DFVNfGjk7QxHlPN/zg8Yi4lRYjvCr7JQkpgAGMjz8scazZr21pPRjIfK7K0H8XC8jXNgJbknWqOIwOejDPcAkP6whmjzlplTXJUq1tdDiE5N65yb7h3eBMqfKq9RsvmKP/dH/ZWOFnoDcfWK19OcToQDhU2XHySUnY9WlcyZ6tnPtW9HQgijXVlf5delmPO+78G9kS40A2UaGU='}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--73c4604d-a9a8-48d5-8938-ccb5fc8a8649-0', tool_calls=[{'name'

In [5]:
response["messages"][-1].content_blocks

[{'type': 'text',
  'text': 'Apple Inc. (AAPL) is currently trading at $277.82.',
  'extras': {'signature': 'CooCAXLI2nyv5x1ymSm/+h04KtVU1aSqH49Qd9Ujq7bHHVlB/nADzCIo8vjEQx3cD/uOsC6b7LmteVbrUeoJENUYwOOSELlAowvXBHVvmB4SF9sWg5yTPvo1C57QhrJnFqIRKJVJQPFCPuUdQBnj/R5rocvS2wM4wSejHLRQ/qjCXwH+aygL9NZqOtaKLF1MJ+A3fKlEfpvopqn3wJDZ2qXt0W5aZRre7on06M2UCOd0Za9u+E3PZRjgySFYtAyh0i4f/zqArAVShkoDwUxlved3fA1DF3YmsqOAjDPdjJGkY3O24ip+DPE7Pp7O+QoVu/4LhDwzjLesJ/jmi0p62/gl0EA43mVCR1Cpwvw='}}]

In [6]:
response = agent.invoke({
    "messages": ["What's Apple's current stock price?"]
})
response

{'messages': [HumanMessage(content="What's Apple's current stock price?", additional_kwargs={}, response_metadata={}, id='743480a4-de86-4f4f-979a-401c4936b52b'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'web_search', 'arguments': '{"query": "Apple stock price"}'}, '__gemini_function_call_thought_signatures__': {'94aaa059-46ad-493c-a8ea-e1b2a467e7ad': 'CpQCAXLI2nyuaiM6Gwv7nurCcQqafYMTLbG/42v9scyzvwQycmKKMsgioxybIVtzFiWl5m21RWEogiy4Qo1wSQPtQvfYTQVkilVeZa4IGl7BOEvxwy8AveLq7HGJOxu/tkDCvdmQ8t4VSWXWNfkAVmsanefyu453mcP1Jaam8RCipPoioLm34vdjhGoKkbUZIhvvDNuq9LmiHQF36ttVlz4y2sYmckcDy/RVqa+lNmtS5ER6UgBsfttXw19RyRYe23nK5GI5z8mYKRrxvxoD4+AWDZadzuFPI9VaIzNcAkZXpr4rHc93pChyM/dxBx1E9cKAyfP5WB0+jqwdnOlo1Ud+4cPGIpDLV4K08AK0h2t5dKi9FomQ'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--958e0768-de86-4091-b172-518824af

## 2. Short-term Memory with SQLite

Add conversation persistence using SQLite checkpointer. Agent remembers previous turns within a session.

In [7]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

conn = sqlite3.connect("data/financial_agent.db", check_same_thread=False)
checkpointer = SqliteSaver(conn)

agent_memory = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    checkpointer=checkpointer
)

In [8]:
config = {"configurable": {"thread_id": "session_1"}}

# First turn
response = agent_memory.invoke({
    "messages": [HumanMessage("Search for Apple's latest earnings")]
}, config)
print(response["messages"][-1].text)

Apple's latest reported earnings, for the fiscal quarter ending December 2023 (Q1 2024), were released on February 1, 2024.

Here's a summary:
*   **Earnings Per Share (EPS):** $2.18 (beating the forecast of $2.10)
*   **Revenue:** $119.58 billion (surpassing the forecast of $118.06 billion)


In [ ]:
from langchain.agents.middleware import SummarizationMiddleware

agent_summary = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    checkpointer=checkpointer,
    middleware=[
        SummarizationMiddleware(
            model="gemini-2.5-flash",
            trigger=[("tokens", 4000), ("messages", 10)],
            keep=("messages", 4)
        )
    ]
)

In [10]:
response

{'messages': [HumanMessage(content="Search for Apple's latest earnings", additional_kwargs={}, response_metadata={}, id='607a3ed5-f0b5-40f8-81cf-fec8fc782221'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'web_search', 'arguments': '{"query": "Apple latest earnings"}'}, '__gemini_function_call_thought_signatures__': {'9e25df14-4089-406e-9bfa-cc57026ab4dc': 'CrwCAXLI2ny/gErxI2LkyBZLm9M4U7voFqrJqLvxRwWOYb7rM1LuKvytCU0XLCsZlrRDPtivofavfH9rSp5rB5h2t75GfjhQBRnEnNrqPoCVki4uyIZBti43bo+c2/wS2mMYa5LOe233sbN7wCTy3orAjvu7dhpCGctZyFVdjD1CkKOr3c+Z/YetKAQVgAJbMJg2TCtajGgQlD2Znhr+DYcfE12CBtmoLn/rVmKWREWt4edhBJyLYJbT5BKKt67kE/7rV2hCLOJlxmDIVGK7GU5m19nOK6pe2MnSesjDMAhKtHv6NtPmIVY8adz9JJUSLTlt7wro3+d7++kazJIO7Ij7KAMc3Lr9JFasjhm+4ZOXRW7551bjnVMqXIBUlDyFoKPgKzKcii8PlFic5wKwp/xOWBFbhgjZy5ZGMms3kw=='}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'g

## Built-in Middleware

### 1. Middleware: Summarization

Automatically summarize old messages when history grows too long.

**Use Case**: Long conversations that exceed context windows.

In [25]:
from langchain.agents.middleware import SummarizationMiddleware

agent_summary = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    checkpointer=checkpointer,
    middleware=[
        SummarizationMiddleware(
            model=model,
            trigger= ("messages", 10),
            keep=("messages", 5)
        )
    ]
)

In [26]:
queries = [
    "Search for Apple stock",
    "What about Microsoft?",
    "Compare their P/E ratios"
]

for query in queries:
    response = agent_summary.invoke({"messages": [HumanMessage(query)]}, config)
    print(f"Q: {query}\nA: {response['messages'][-1].content}\n")

state = agent_summary.get_state(config)
print(f"Total messages: {len(state.values['messages'])}")

Q: Search for Apple stock
A: [{'type': 'text', 'text': "Apple's (AAPL) P/E ratio (TTM) is 37.29. Which other company's P/E ratio would you like to compare it with?", 'extras': {'signature': 'CukCAXLI2nyeiZQX8sjG6Vd7BViehVLqG8KkSJT8cuxITk+QqclITaZGzrgAl4V//YXdhE2Vt1EvO3ehvaKiQVoIMZIKdbqAkPX2r1AKuvcOhAq7pCZNFykfgU96vfc6CPc6+dE3faQsaZ2v1r0zKWXp+3ynpIBicQYR4UkoleZ6cn47Wt6GtRjDl6Ow8DkTyHWITi8kE4YtwjeDlSzFVvkxYcA3VLsJMc3hxJNNsqR6hyqMCQj9o2CqAq0DQXYVcE4Wic9G+ImtlIy12b3eVIStrwLPzQxnRM+C434DypR1ZRBMN+0fwkhh48l8UYvDZfxwsxCw86rxKMsF483KNGCyoB2Q5Nh6AnB9y2ZrQMMUQJncwLKsb0RxIUhRZZV4IyfAf7L54j+prr0I1cgbBFRdofeV7qqNnpcEoOsfKSu/CqUfRPCVSG5h28dPmf49vTPN619qNpdHn8mImXNh/yeJoXIwqB6NYgZxZQ=='}}]

Q: What about Microsoft?
A: Microsoft's (MSFT) P/E ratio (TTM) is 38.55.

Therefore:
*   **Microsoft (MSFT): 38.55**
*   **Apple (AAPL): 37.29**

Based on these trailing twelve-month (TTM) P/E ratios, Microsoft has a slightly higher P/E ratio than Apple.

Q: Compare their P/E ratios
A: [{'type': 'text', 'text': "A

In [27]:
response

{'messages': [HumanMessage(content='Here is a summary of the conversation to date:\n\nThe user wants to compare the P/E ratios of Microsoft and Apple.', additional_kwargs={}, response_metadata={}, id='cad77586-3c36-4956-a0b3-e63f2676cfa2'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'web_search', 'arguments': '{"query": "Apple stock"}'}, '__gemini_function_call_thought_signatures__': {'17f41177-7040-47d7-8d68-16770c48c05e': 'Cs8BAXLI2nytxZlFbjy6RCCbl53V7fa9x2psHLUtiW0R67t6mCqW8cNyCzSKjPZxUZ6RlxUrYk98Hb3pGExi7Nrp2JPCHrGSNN0QE4KBtWQx9JyZuLw2il98tzHALANBTPRUZLEVO5O71FwyZDUzjiylsB0ze3ZrL8DFAZmwHbA5t+u9PHWdVKxL6gLBkP/JJoz3nLr0E9zwg9Av22IVh5gAQS/zb+6X5yR0ALwBM5ysR2GKTv9YlK5+It6FSIJBnxFB/bDf75l+xvoizdOj4g5d'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--50f7f723-7919-43b5-af0e-0107990b82af-0', tool_calls

## 4. Middleware: Model Call Limit

Limit the number of model calls per request to prevent runaway costs.

**Exit Behaviors**: `"end"` (stop) or `"continue"` (proceed without model)

In [ ]:
from langchain.agents.middleware import ModelCallLimitMiddleware
from langchain.agents.middleware import ToolCallLimitMiddleware
from langchain.agents.middleware import ModelFallbackMiddleware

agent_limit = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    middleware=[
        ModelCallLimitMiddleware(
            run_limit=2,
            exit_behavior="end"
        ),
        ToolCallLimitMiddleware(
            run_limit=2,
            exit_behavior="continue"
            
        ),
                ModelFallbackMiddleware(ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite"))
    ]
)

response = agent_limit.invoke({
    "messages": [HumanMessage("what is the latest apple stock price and what is the latest weather in mumbai?")]
})
print("Limited calls:", response["messages"][-1].text)

Limited calls: The latest Apple stock price is $279.00, which is up $0.22 (+0.08%) in after-hours trading.

The current weather in Mumbai is 29°C (84.2°F) and overcast. The last update was on 2025-12-08 22:15.


## 7. Middleware: Guardrails and PII Detection

Automatically detect and redact/mask personally identifiable information.

**Strategies**: `"redact"` (remove), `"mask"` (replace with ***)

In [35]:
from langchain.agents.middleware import PIIMiddleware

agent_pii = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    middleware=[
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        PIIMiddleware("credit_card", strategy="mask", apply_to_input=True),
        PIIMiddleware("api_key", detector=r"sk-[a-zA-Z0-9]{32}", strategy="block", apply_to_input=True)
    ]
)

response = agent_pii.invoke({
    "messages": [HumanMessage("Hi my name is laxmi kant tiwari and my email is test@example.com for Apple updates")]
})
print(response["messages"][-1].text)

Hi Laxmi,

Thank you for your interest in Apple updates. As an AI, I cannot directly subscribe you to email updates or store your personal information.

To receive official Apple updates, please visit the official Apple website and sign up for their newsletters or notifications directly.


## 8. Middleware: Todo List

Track and manage multi-step tasks within the agent.

In [36]:
from langchain.agents.middleware import TodoListMiddleware

agent_todo = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    checkpointer=checkpointer,
    middleware=[TodoListMiddleware()]
)

config = {"configurable": {"thread_id": "todo_session"}}

response = agent_todo.invoke({
    "messages": [HumanMessage("Analyze Apple revenue and compare competitors")]
}, config)
print(response["messages"][-1].text)

Here's a data-driven analysis of Apple's revenue and a comparison with its key competitors for the quarter ended September 30, 2025:

**Apple's Revenue Performance:**
Apple reported a strong fiscal 2025 fourth quarter, with **$102.5 billion** in quarterly revenue. This included a September quarter revenue record for iPhone and an all-time revenue record for Services, highlighting the continued strength of its core hardware product and the significant growth of its ecosystem-driven services. For the full fiscal year 2025, Apple's revenue reached $416 billion.

**Competitor Revenue Comparison:**

To provide a comprehensive comparison, I analyzed the latest quarterly revenues of Apple's major competitors across its various segments:

*   **Alphabet (Google):** Reported consolidated revenues of **$102.3 billion** for the quarter, placing it in a very similar revenue bracket as Apple. Google Services (including Search, YouTube ads, Android, and devices) contributed $87.1 billion, while Goog

## 10. Context Engineering: Delete and Trim Messages

Programmatically trim conversation history to save tokens.

## 11. Combined Middleware

Stack multiple middleware for production-ready agents.

In [ ]:
agent_complete = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    checkpointer=checkpointer,
    middleware=[
        SummarizationMiddleware(model="gemini-2.5-flash", trigger={"messages": 12}, keep={"messages": 5}),
        ModelCallLimitMiddleware(run_limit=5),
        ToolCallLimitMiddleware(run_limit=3),
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        TodoListMiddleware()
    ]
)

## Advanced Features

## 12. Structured Output

Return type-safe Pydantic models from agent responses.

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional

class FinancialAnalysis(BaseModel):
    """Structured financial analysis."""
    company: str = Field(description="Company name")
    stock_symbol: str = Field(description="Stock ticker")
    current_price: Optional[str] = Field(description="Current price")
    analysis: str = Field(description="Brief analysis")
    recommendation: str = Field(description="Buy/Hold/Sell")

In [ ]:
model_structured = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
model_with_tools = model_structured.bind_tools([base_tools.web_search])
structured_llm = model_with_tools.with_structured_output(FinancialAnalysis)

agent_structured = create_agent(
    model=structured_llm,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt="Provide structured financial analysis.",
    checkpointer=checkpointer
)

In [ ]:
config = {"configurable": {"thread_id": "structured_session"}}

response = agent_structured.invoke({
    "messages": [HumanMessage("Analyze Apple stock")]
}, config)

if "structured_response" in response:
    analysis = response["structured_response"]
    print(f"Company: {analysis.company}")
    print(f"Symbol: {analysis.stock_symbol}")
    print(f"Analysis: {analysis.analysis}")
    print(f"Recommendation: {analysis.recommendation}")

## Streaming Modes

Three streaming modes for real-time agent updates:
- **`messages`**: Stream individual messages as they're generated
- **`updates`**: Stream state updates after each step
- **`values`**: Stream complete state values

## 13. Streaming: Messages Mode

Stream messages token-by-token as they're generated.

In [ ]:
agent_stream = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    checkpointer=checkpointer
)

config = {"configurable": {"thread_id": "stream_1"}}

print("\n=== Stream: Messages ===")
for chunk in agent_stream.stream({
    "messages": [HumanMessage("Quick Apple stock update")]
}, config):
    print(chunk)
    print("---")

## 14. Streaming: Updates Mode

Stream state updates after each agent step (tool calls, model responses).

In [ ]:
config = {"configurable": {"thread_id": "stream_2"}}

print("\n=== Stream: Updates ===")
for chunk in agent_stream.stream({
    "messages": [HumanMessage("Tech sector outlook?")]
}, config, stream_mode="updates"):
    print(chunk)
    print("---")

## 15. Streaming: Values Mode

Stream complete state snapshots (useful for monitoring message count).

In [ ]:
config = {"configurable": {"thread_id": "stream_3"}}

print("\n=== Stream: Values ===")
for chunk in agent_stream.stream({
    "messages": [HumanMessage("Compare Apple and Microsoft")]
}, config, stream_mode="values"):
    if "messages" in chunk:
        print(f"State: {len(chunk['messages'])} messages")
    print("---")